In [1]:
import os

In [2]:
pwd

'd:\\cnn-example\\cnn-chickendisease-example\\research'

In [3]:
os.chdir("../")

In [4]:
pwd

'd:\\cnn-example\\cnn-chickendisease-example'

In [ ]:
# now we dont need to update config.yaml here
# for model evaluation here, no need
# directly define the entity - path etc,

In [27]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    #mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [28]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [29]:
class ConfigurationManager:
    '''
    creates a class that will manage all your configuration settings - 
    think of it as a control center for your model evaluation settings.
    '''
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH, 
        params_filepath = PARAMS_FILE_PATH): 
        self.config = read_yaml(config_filepath) #  reads your configuration file (in YAML format) and stores all its settings in self.config. It's like opening a recipe book and keeping it handy.
        self.params = read_yaml(params_filepath) # Similarly, this reads your parameters file and stores settings in self.params. These are specific settings like image size and batch size that your model needs.

        create_directories([self.config.artifacts_root]) 
        # This ensures your main storage folder (artifacts) exists.
        # If it doesn't, it creates it - like making sure you have a filing cabinet before you try to store files.
        

    
    def get_evaluation_config(self) -> EvaluationConfig:
        '''
         responsible for creating a special package of settings just for model evaluation. 
         The -> EvaluationConfig part promises it will return an EvaluationConfig object.
        '''
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5", # This specifies where your trained model is saved - it's looking for a file named "model.h5" in your training folder.
            training_data="artifacts/data_ingestion/chickenfecalimages",
            all_params=self.params, # This passes along all your model parameters at once - it's like handing over the entire instruction manual.
            params_image_size=self.params.IMAGE_SIZE, #This specifies how big your images need to be (like 224x224 pixels) - it must match what your model expects.
            params_batch_size=self.params.BATCH_SIZE # This sets how many images will be processed at once during evaluation - like deciding how many dishes to cook in one go.
        )
        return eval_config

In [30]:
import tensorflow as tf
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [31]:
# import the model
model = tf.keras.models.load_model("artifacts/training/model.h5")

# This is like opening up your trained CNN from storage, making it ready for testing.

In [32]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config # This initializes your evaluation setup using the configuration

    
    def _valid_generator(self):
        '''
        This sets up how your images will be prepared. 
        The rescale=1./255 converts your image pixels from 0-255 to 0-1 (like converting dollars to cents for easier calculation), 
        and validation_split=0.30 reserves 30% of your data for validation.
        '''

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )
        
        '''
        The below defines how images will be processed in batches. 
        The target_size ensures all images are the same size,
        and bilinear interpolation helps resize images smoothly.
        '''

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        


        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            "This creates your image processing pipeline using the settings we defined."
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            "This sets up a stream of images from your directory, specifically using the validation subset."
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    # is marked as a static method because loading a model from a path doesn't require any instance-specific information
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        "function to load the model"
        return tf.keras.models.load_model(path)
    

    def evaluation(self): 
        '''
        
        This loads your model, prepares your validation data, and calculates how well your model performs.
         Finally, you save the results:
        '''
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    
    def save_score(self):
        "saving in scores variable"
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        # saving the scores output in json file
        save_json(path=Path("scores.json"), data=scores)

In [33]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.save_score()
    

except Exception as e:
   raise e

# in the left pane below, you can find scores.json file
# and then convert this notebook into modular piece.

[2025-01-02 12:31:04,964: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-02 12:31:04,967: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-02 12:31:04,970: INFO: common: created directory at: artifacts]
Found 14 images belonging to 2 classes.
1/1 [==============================] - 3s 3s/step - loss: 27.6701 - accuracy: 0.5000
[2025-01-02 12:31:08,780: INFO: common: json file saved at: scores.json]
